In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from string import punctuation
import re
from collections import Counter
from sklearn.metrics import accuracy_score

C:\Users\kuzne\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

сайт для положительных отзывов защищен, пришлось использовать селениум

In [3]:
driver = webdriver.Chrome()

In [4]:
page = driver.get("https://cmp24.ru/otzyv/18-chayniki-elektricheskie/?p=3")

In [5]:
good = []

In [6]:
for page_number in range(5, 10):
    url = f'https://cmp24.ru/otzyv/18-chayniki-elektricheskie/?p={page_number}'
    req = driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    rating = [int(i.text) for i in soup.find_all('span', {'itemprop': 'ratingValue'})[1:]]
    description = [i.text for i in soup.find_all('p', {'itemprop': "description"})]
    both = dict(zip(description, rating))
    for k, v in both.items():
        if v > 3:
            good.append(k)

накраулерила хороших отзывов

In [7]:
url = f'https://nanegative.ru/termopot-tesler-tp-5055-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
nedostatki = [i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})]

In [8]:
url = f'https://nanegative.ru/braun-wk-300-2011-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
nedostatki.extend([i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})])

In [9]:
url = f'https://nanegative.ru/xiaomi-mi-kettle-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
nedostatki.extend([i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})])

In [10]:
url = f'https://nanegative.ru/homestar-hs-1010-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
nedostatki.extend([i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})])

In [11]:
url = f'https://nanegative.ru/redmond-skykettle-g210s-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
nedostatki.extend([i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})])

собрала плохие отзывы (на этом сайте в основном негативные)

In [12]:
def filter_s(i):
    if i != ' —':
        return True
    else:
        return False

bad = list(filter(filter_s, nedostatki))

In [13]:
good = list(filter(filter_s, good))

отфильтровала пустые отзывы

In [14]:
morph = MorphAnalyzer()

In [15]:
bad_text = ' '.join(bad[:114])
good_text = ' '.join(good[:114])

In [16]:
tokens = [token for token in word_tokenize(bad_text)]
tokens_lower = [token.lower() for token in tokens]

In [17]:
from nltk.corpus import stopwords
sw = stopwords.words('russian')
filtered = [w for w in tokens_lower if w not in sw]

In [18]:
lemmas = [morph.parse(token)[0].normal_form for token in filtered]

In [19]:
import string
punct = set(string.punctuation)
def filter_l(i):
    if i not in punct:
        return True
    else:
        return False
    
lemmas_w_p = list(filter(filter_l, lemmas))

#но не все можно таким образом выкинуть, поэтому придется сделать регулярки :/

for smth in lemmas_w_p:
    if re.match(r'''.*(\.|[0-9]|'|"|`).*''', smth) is not None:
        lemmas_w_p.remove(smth)

In [20]:
lemmas_w_p

['интересный',
 'предмет',
 'интерьер',
 'назначение',
 'использовать',
 'долго',
 'получиться',
 'нравиться',
 'обращаться',
 'аккуратно',
 'ровно',
 'год',
 'дисплей',
 'стать',
 'мерцаать',
 'выключиться',
 'включаться',
 'прочитать',
 'положительный',
 'отзыв',
 'данный',
 'термопот',
 'tesler',
 'приобрести',
 'начало',
 'апрель',
 'год',
 'очень',
 'довольный',
 'первый',
 'число',
 'декабрь',
 'перестать',
 'подавать',
 'вода',
 'отвезти',
 'сервис',
 'вернуть',
 'мы',
 'новогодний',
 'праздник',
 'начало',
 'февраль',
 'перестать',
 'подавать',
 'вода',
 'отвезти',
 'сервис',
 'пройти',
 'месяц',
 'мы',
 'выдать',
 'акт',
 'неремонтопригодность',
 'написать',
 'ремонт',
 'невозможный',
 'дефект',
 'блок',
 'отдать',
 'должное',
 'ozon',
 'следующий',
 'день',
 'вернуть',
 'деньга',
 'озон',
 'счёт',
 'свой',
 'отзыв',
 'разместить',
 'смочь',
 'писать',
 'приобретать',
 'данный',
 'товар',
 'бывать',
 'посоветовать',
 'никто',
 'брать',
 'модель',
 'хотеть',
 'проблемма',
 'здо

сделала токенизацию, лемматизацию, привела текст к нижнему регистру, убрала цифры/пунктуацию

In [21]:
lemmas_c = Counter(lemmas_w_p)

In [22]:
evil_lemmas = {lemma for lemma in lemmas_c.keys() if lemmas_c[lemma] > 1}

In [23]:
tokens = [token for token in word_tokenize(good_text)]
tokens_lower = [token.lower() for token in tokens]
filtered = [w for w in tokens_lower if w not in sw]
lemmas = [morph.parse(token)[0].normal_form for token in filtered]
lemmas_w_p = list(filter(filter_l, lemmas))

#но не все можно таким образом выкинуть, поэтому придется сделать регулярки :/

for smth in lemmas_w_p:
    if re.match(r'''.*(\.|[0-9]|'|"|`).*''', smth) is not None:
        lemmas_w_p.remove(smth)

lemmas_c2 = Counter(lemmas_w_p)
nice_lemmas = {lemma for lemma in lemmas_c2.keys() if lemmas_c2[lemma] > 1}

In [24]:
only_good = nice_lemmas - evil_lemmas
only_good

{'-термометр',
 '.чайник',
 '1',
 '21',
 'fort',
 'kit',
 'kitfort',
 'абсолютно',
 'автоматически',
 'автоматический',
 'аргументировать',
 'безопасность',
 'белый',
 'бить',
 'благодаря',
 'большой',
 'бытовой',
 'бюджет',
 'важно',
 'ваш',
 'вверх',
 'вежливый',
 'век',
 'великий',
 'вес',
 'весьма',
 'вечер',
 'видеть',
 'вместе',
 'внешне',
 'внимание',
 'возникать',
 'восторг',
 'впечатление',
 'вписаться',
 'вполне',
 'вращаться',
 'встроить',
 'всякий',
 'вт',
 'выбирать',
 'выглядеть',
 'выдача',
 'выделяться',
 'выдержать',
 'выключение',
 'высокий',
 'выше',
 'выясниться',
 'газовый',
 'гамма',
 'главное',
 'глаз',
 'голубой',
 'горлышко',
 'графит',
 'графитовый',
 'греться',
 'громкий',
 'гурман',
 'далёкий',
 'деталь',
 'детский',
 'дизайн',
 'диск',
 'дисковый',
 'дно',
 'добросовестный',
 'добрый',
 'должный',
 'доставлять',
 'достаточно',
 'достать',
 'достойный',
 'ежевичный',
 'журчание',
 'жёсткость',
 'заваривать',
 'заварить',
 'заварка',
 'зайти',
 'заказывать',


In [25]:
only_bad = evil_lemmas - nice_lemmas
only_bad

{'6',
 'bosch',
 'braun',
 'cегодень',
 'ozon',
 'redmond',
 'tesler',
 'xiaomi',
 'акт',
 'акция',
 'алиса',
 'апрель',
 'база',
 'беречь',
 'биметаллический',
 'близкий',
 'блок',
 'больший',
 'брак',
 'браковать',
 'браун',
 'бренд',
 'быть',
 'вернуть',
 'ветер',
 'вещь',
 'взять',
 'видимо',
 'вилка',
 'включиться',
 'внезапно',
 'внести',
 'водичка',
 'возвращать',
 'вонючий',
 'вонять',
 'вопрос',
 'вскипятить',
 'второй',
 'вывести',
 'вывод',
 'выдать',
 'выкинуть',
 'выключатель',
 'выключаться',
 'выключить',
 'гарантия',
 'главный',
 'глюк',
 'группа',
 'давать',
 'дача',
 'два',
 'дебаты',
 'декабрь',
 'делать',
 'дело',
 'дефект',
 'дешёвый',
 'дисплей',
 'дистанционный',
 'днище',
 'долгий',
 'должное',
 'доп',
 'дополнительный',
 'ещё',
 'жаль',
 'заваха',
 'зависеть',
 'заводский',
 'заливать',
 'залипать',
 'залипнуть',
 'заново',
 'защита',
 'звук',
 'игра',
 'изделие',
 'изменение',
 'иметь',
 'инструкция',
 'интересный',
 'исходить',
 'итог',
 'кассовый',
 'категор

нашла слова, встречающиеся только в + или - отзывах

дальше наберем штук 30 отзывов для определения + или - отзыв

In [26]:
all_together1 = []
url = f'https://nanegative.ru/philips-hd9340-otzivy'
req = driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
all_together1.extend([i.text for i in soup.find_all('td', {'itemprop': 'reviewBody'})])
all_together1

['Однозначно советую Филипсы.\nUPDATE. Спустя 6 месяцев перестал греть воду и сервисным центром был признан неремонтопригодным. Обменял на аналогичный по стоимости Филипс... буду пробовать.',
 'либо попалась бракованная партия , либо  эта кнопка вообще слабое место у данной модели, но впечатления от пользования и  двух обменов  негативное.',
 'До этого чайника и у меня, и у моей мамы был VITEK VT-1102. Стеклянный. Испарины никогда не было. \nРасстраивает, что купила я его, чтобы порадовать маму. И вручила заранее, как подарочек от Деда Мороза. А чайник ни только не радует, а даже раздражает!.. За немалые деньги для такого предмета приобрела ухудшитель настроения!.. В первые в жизни пишу отрицательный отзыв. И говорю всем: не берите такой чайник!',
 'несмотря на отзывы остальных, могу сказать из собственного опыта, что вода пахнет пластиком от дешевой крышки. Брать не советую. Сам попробовал несколько раз прокипятить с лимонной кислотой, не помогло, на следующий день сдал, буду искать ч

In [27]:
l1 = [i*-1 for i in [1]*len(all_together1)]

In [28]:
all_together2 = []
for page_number in range(11, 13):
    url = f'https://cmp24.ru/otzyv/18-chayniki-elektricheskie/?p={page_number}'
    req = driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    rating = [int(i.text) for i in soup.find_all('span', {'itemprop': 'ratingValue'})[1:]]
    description = [i.text for i in soup.find_all('p', {'itemprop': "description"})]
    both = dict(zip(description, rating))
    for k, v in both.items():
        if v > 3:
            all_together2.append(k)

In [29]:
l2 = [1]*len(all_together2)

In [30]:
l3 = l1 + l2

In [31]:
l = [int(i) for i in l3][:30]

In [32]:
all_together1.extend(all_together2)

In [33]:
test = all_together1[:30]

In [34]:
def to_tokenize(my_text):
    tokens = [token for token in word_tokenize(my_text)]
    tokens_lower = [token.lower() for token in tokens]
    filtered = [w for w in tokens_lower if w not in sw]
    lemmas = [morph.parse(token)[0].normal_form for token in filtered]
    lemmas_w_p = list(filter(filter_l, lemmas))
    for smth in lemmas_w_p:
        if re.match(r'''.*(\.|[0-9]|'|"|`).*''', smth) is not None:
            lemmas_w_p.remove(smth)
    c = 0
    for j in lemmas_w_p:
        if j in only_bad:
            c -= 1
        elif j in only_good:
            c += 1
    if c > 0:
        return 1
    elif c == 0:
        return -1 #для упрощения считаю нейтральные отзывы ближе к плохим
    else:
        return -1

In [35]:
test_y = [int(to_tokenize(i)) for i in test]

In [36]:
score = accuracy_score(l, test_y)
score

0.8333333333333334

сначала скачивала - отзывы и создавала список с -1 равный по длине списку с - отзывами, для + отзывов то же самое, потом соединила эти списки в один, потом сделала функцию для подсчета положительно или негативно окрашенных слов, ее результаты сравнила, аккураси очень даже неплохая...

предложения по **улучшению**:

    - умнее делать парсинг - может, засунуть его в функции, я не успела это сделать :(
    
    - ну и как-то улучшить проверку на положительность/отрицательность, например, не создавать список по длине скачанных отзывов с сайта с негативными а потом добавлять с негативными, а делать через словарь

предложения по тому, что **еще можно посмотреть**:
    
    - посмотреть процентное соотношение плохих/хороших слов в отзыве, посмотреть среднее по хорошим, и по плохим
    
    - сгенерировать прототипические универсальные плохие и хорошие отзывы